In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation
from tensorflow.python.keras import metrics
from sklearn.preprocessing import StandardScaler
from tensorflow.python import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk

# 파일읽기

In [2]:
df=pd.read_excel('../optimal_data3_1/'+'Continous_2weeks_2day_1term.xlsx')

In [3]:
df

,0,1,2,3,4,5,6,7,8
0,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
1,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
2,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
3,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
4,#AAGQKY,45.15,20.612673,72.0,0.0,0.0,0.0,148.0,0
...,...,...,...,...,...,...,...,...,...
58820,#ZZLSSL,68.00,29.048656,0.0,0.0,0.0,0.0,153.0,0
58821,#ZZLSSL,68.00,29.048656,0.0,0.0,0.0,0.0,153.0,0
58822,#ZZLSSL,68.00,29.048656,0.0,0.0,0.0,0.0,153.0,0
58823,#ZZLSSL,68.00,29.048656,0.0,0.0,0.0,0.0,153.0,0


# 데이터를 필요한 부분으로 X, Y 나눠 읽어오기

In [4]:
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]
pred_list=[]

In [5]:
#한 사람당 데이터 수
Count_1=int(181*0.1)
#한 사람당 데이터 수
Count_2=181

X_test=pd.DataFrame()
X_train=pd.DataFrame()
y_test=pd.DataFrame()
y_train=pd.DataFrame()
empty=pd.DataFrame()

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]

# 모델

## Deep Neural Network

## Input Feature : 6

## Hidden Layer : 128 - 256 - 128

## optimizer : adam, loss : categorical_crossentropy

In [6]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
	keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
	keras.layers.Dense(3, activation=tf.nn.softmax),
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
for i in range(10):
    X_test=empty
    X_train=empty
    y_test=empty
    y_train=empty
    y_test_list=[]
    for j in range(322):
        X_temp_test=X.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        X_test=pd.concat([X_test,X_temp_test])
        X_temp_train=X.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        X_train=pd.concat([X_train,X_temp_train])
        
        y_temp_test=y.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        y_test=pd.concat([y_test,y_temp_test])
        y_temp_train=y.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        y_train=pd.concat([y_train,y_temp_train])
        
    
    print('SMOTE 적용 전 Train 레이블 값 분포: \n', y_train.value_counts())
    print('SMOTE 적용 전 Test 레이블 값 분포: \n', y_test.value_counts())
    
    # SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train,y_train)
#     X_test,y_test = smote.fit_resample(X_test,y_test)
    print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_test.shape, y_test.shape)
    print('SMOTE 적용 후 Train 레이블 값 분포: \n', y_train.value_counts())
    print('SMOTE 적용 후 Test 레이블 값 분포: \n', y_test.value_counts())
    
    # 원핫인코딩
    # 예시 : 1 , 2 -> (1,0) , (0,1)
    y_train=pd.get_dummies(y_train[0])
    y_test=pd.get_dummies(y_test[0])
    
    history=model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=18)
    predicted=model.predict(X_test)
    results = model.evaluate(X_test, y_test, batch_size=8)
    print("[{}]Accuracy : {}".format(i,results[1]))   
    
    predicted=pd.DataFrame(predicted)
    predicted=predicted.idxmax(axis=1)
    y_test=y_test.idxmax(axis=1)
    
    print("predicted",predicted)
    print("y_test",y_test)
    
    #f1score
    f1 = f1_score(y_test,predicted, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    p_rlist=sk(y_test,predicted,average='weighted')
    print("[{}]Precision : {}".format(i,p_rlist[0]))
    print("[{}]Recall : {}".format(i,p_rlist[1]))
    print()
     #결과 배열에 넣기
    Result[i][0]=results[1]
    Result[i][1]=f1
    Result[i][2]=p_rlist[0]
    Result[i][3]=p_rlist[1]
    del results
    del f1
    del p_rlist

SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    49915
1.0     1621
2.0      950
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    5199
1.0     368
2.0     229
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (5796, 6) (5796, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    49915
1.0    49915
2.0    49915
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    5199
1.0     368
2.0     229
dtype: int64
Epoch 1/100
8320/8320 [==============================] - 30s 3ms/step - loss: 0.8174 - accuracy: 0.6058 - val_loss: 0.8764 - val_accuracy: 0.6140
Epoch 2/100
8320/8320 [==============================] - 30s 4ms/step - loss: 0.6042 - accuracy: 0.7426 - val_loss: 0.9955 - val_accuracy: 0.6177
Epoch 3/100
8320/8320 [==============================] - 29s 4ms/step - loss: 0.5130 - accuracy: 0.7890 - val_loss: 1.0264 - val_accuracy: 0.6323
Epoch 4/100
8320/8320 [==============================] - 30s 4ms/step - loss: 0.4609 - accuracy: 0.8130 - val_loss: 0.9995 - val_accuracy: 0.6767
Epoch 5/100
8320/8320 [==========================

8320/8320 [==============================] - 28s 3ms/step - loss: 0.2086 - accuracy: 0.9187 - val_loss: 1.5727 - val_accuracy: 0.7181
Epoch 55/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.2072 - accuracy: 0.9196 - val_loss: 1.5246 - val_accuracy: 0.7288
Epoch 56/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.2057 - accuracy: 0.9202 - val_loss: 1.5687 - val_accuracy: 0.7220
Epoch 57/100
8320/8320 [==============================] - 28s 3ms/step - loss: 0.2050 - accuracy: 0.9202 - val_loss: 1.5557 - val_accuracy: 0.7379
Epoch 58/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.2035 - accuracy: 0.9212 - val_loss: 1.5732 - val_accuracy: 0.7279
Epoch 59/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.2038 - accuracy: 0.9213 - val_loss: 1.5985 - val_accuracy: 0.7274
Epoch 60/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.2022 - accuracy: 0.9214 - val_loss: 1.6154 - val_a

8320/8320 [==============================] - 28s 3ms/step - loss: 0.1755 - accuracy: 0.9319 - val_loss: 0.2787 - val_accuracy: 0.8944
Epoch 4/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.1742 - accuracy: 0.9328 - val_loss: 0.2993 - val_accuracy: 0.8853
Epoch 5/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.1752 - accuracy: 0.9323 - val_loss: 0.3285 - val_accuracy: 0.8791
Epoch 6/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.1738 - accuracy: 0.9334 - val_loss: 0.3022 - val_accuracy: 0.8891
Epoch 7/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.1740 - accuracy: 0.9336 - val_loss: 0.2775 - val_accuracy: 0.8898
Epoch 8/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1725 - accuracy: 0.9332 - val_loss: 0.2606 - val_accuracy: 0.8935
Epoch 9/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.1727 - accuracy: 0.9335 - val_loss: 0.2970 - val_accurac

Epoch 59/100
8320/8320 [==============================] - 28s 3ms/step - loss: 0.1576 - accuracy: 0.9399 - val_loss: 0.2584 - val_accuracy: 0.9086
Epoch 60/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.1576 - accuracy: 0.9399 - val_loss: 0.2821 - val_accuracy: 0.8961
Epoch 61/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.1588 - accuracy: 0.9398 - val_loss: 0.2180 - val_accuracy: 0.9220
Epoch 62/100
8320/8320 [==============================] - 28s 3ms/step - loss: 0.1572 - accuracy: 0.9404 - val_loss: 0.2845 - val_accuracy: 0.9023
Epoch 63/100
8320/8320 [==============================] - 28s 3ms/step - loss: 0.1573 - accuracy: 0.9398 - val_loss: 0.2665 - val_accuracy: 0.9013
Epoch 64/100
8320/8320 [==============================] - 28s 3ms/step - loss: 0.1572 - accuracy: 0.9405 - val_loss: 0.2682 - val_accuracy: 0.9017
Epoch 65/100
8320/8320 [==============================] - 27s 3ms/step - loss: 0.1558 - accuracy: 0.9406 - val_loss: 0

8320/8320 [==============================] - 26s 3ms/step - loss: 0.1512 - accuracy: 0.9429 - val_loss: 0.2553 - val_accuracy: 0.9065
Epoch 9/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1496 - accuracy: 0.9430 - val_loss: 0.2689 - val_accuracy: 0.8982
Epoch 10/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1505 - accuracy: 0.9426 - val_loss: 0.2461 - val_accuracy: 0.9122
Epoch 11/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1508 - accuracy: 0.9429 - val_loss: 0.2251 - val_accuracy: 0.9127
Epoch 12/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1502 - accuracy: 0.9431 - val_loss: 0.2291 - val_accuracy: 0.9127
Epoch 13/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1482 - accuracy: 0.9438 - val_loss: 0.2316 - val_accuracy: 0.9055
Epoch 14/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1499 - accuracy: 0.9428 - val_loss: 0.2170 - val_ac

...

Epoch 89/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1749 - accuracy: 0.9358 - val_loss: 0.1724 - val_accuracy: 0.9348
Epoch 90/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1764 - accuracy: 0.9372 - val_loss: 0.1486 - val_accuracy: 0.9489
Epoch 91/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1850 - accuracy: 0.9329 - val_loss: 0.1442 - val_accuracy: 0.9401
Epoch 92/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1768 - accuracy: 0.9368 - val_loss: 0.1218 - val_accuracy: 0.9539
Epoch 93/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1796 - accuracy: 0.9356 - val_loss: 0.1364 - val_accuracy: 0.9460
Epoch 94/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1763 - accuracy: 0.9361 - val_loss: 0.1659 - val_accuracy: 0.9388
Epoch 95/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1845 - accuracy: 0.9344 - val_loss: 0

8320/8320 [==============================] - 26s 3ms/step - loss: 0.1798 - accuracy: 0.9350 - val_loss: 0.1846 - val_accuracy: 0.9341
Epoch 39/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1835 - accuracy: 0.9340 - val_loss: 0.1720 - val_accuracy: 0.9281
Epoch 40/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.2282 - accuracy: 0.9327 - val_loss: 0.4406 - val_accuracy: 0.8318
Epoch 41/100
8320/8320 [==============================] - 25s 3ms/step - loss: 25.4279 - accuracy: 0.9126 - val_loss: 0.1692 - val_accuracy: 0.9350
Epoch 42/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1850 - accuracy: 0.9343 - val_loss: 0.1632 - val_accuracy: 0.9384
Epoch 43/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1867 - accuracy: 0.9338 - val_loss: 0.1795 - val_accuracy: 0.9229
Epoch 44/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1814 - accuracy: 0.9344 - val_loss: 0.1913 - val_

Epoch 94/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1850 - accuracy: 0.9333 - val_loss: 0.1920 - val_accuracy: 0.9149
Epoch 95/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.2450 - accuracy: 0.9294 - val_loss: 0.1494 - val_accuracy: 0.9382
Epoch 96/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1881 - accuracy: 0.9332 - val_loss: 0.1841 - val_accuracy: 0.9282
Epoch 97/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1871 - accuracy: 0.9328 - val_loss: 0.1951 - val_accuracy: 0.9225
Epoch 98/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1873 - accuracy: 0.9317 - val_loss: 0.2007 - val_accuracy: 0.9232
Epoch 99/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1901 - accuracy: 0.9327 - val_loss: 0.2076 - val_accuracy: 0.9061
Epoch 100/100
725/725 [==============================] - 1s 2ms/step - loss: 0.1925 - accuracy: 0.9186
[8]Accuracy : 0

8320/8320 [==============================] - 26s 3ms/step - loss: 0.1896 - accuracy: 0.9311 - val_loss: 0.2045 - val_accuracy: 0.9215
Epoch 44/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1911 - accuracy: 0.9309 - val_loss: 0.2187 - val_accuracy: 0.9284
Epoch 45/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1923 - accuracy: 0.9306 - val_loss: 0.1700 - val_accuracy: 0.9172
Epoch 46/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1922 - accuracy: 0.9312 - val_loss: 0.1892 - val_accuracy: 0.9218
Epoch 47/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1961 - accuracy: 0.9301 - val_loss: 0.1910 - val_accuracy: 0.9222
Epoch 48/100
8320/8320 [==============================] - 25s 3ms/step - loss: 0.1983 - accuracy: 0.9284 - val_loss: 0.1538 - val_accuracy: 0.9375
Epoch 49/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1925 - accuracy: 0.9301 - val_loss: 0.3048 - val_a

Epoch 99/100
8320/8320 [==============================] - 26s 3ms/step - loss: 0.1940 - accuracy: 0.9297 - val_loss: 0.1874 - val_accuracy: 0.9262
Epoch 100/100
725/725 [==============================] - 1s 2ms/step - loss: 0.1633 - accuracy: 0.9453
[9]Accuracy : 0.9453071355819702
predicted 0       0
1       0
2       0
3       0
4       0
       ..
5791    0
5792    0
5793    0
5794    0
5795    0
Length: 5796, dtype: int64
y_test 162      0.0
163      0.0
164      0.0
165      0.0
166      0.0
        ... 
58276    0.0
58277    0.0
58278    0.0
58279    0.0
58280    0.0
Length: 5796, dtype: float64
[9]F1score : 0.9602028124560068
[9]Precision : 0.984637412399001
[9]Recall : 0.9453071083505866



In [8]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,0.739821,0.783108,0.839006,0.739821
1,0.902519,0.914008,0.941008,0.902519
2,0.920980,0.932208,0.956646,0.920980
3,0.924086,0.935778,0.961137,0.924086
4,0.932540,0.943391,0.965826,0.932540
5,0.940131,0.956403,0.982325,0.940131
6,0.939268,0.954250,0.979201,0.939268
7,0.928744,0.947957,0.979468,0.928744
8,0.918564,0.940113,0.975294,0.918565
9,0.945307,0.960203,0.984637,0.945307


In [9]:
print("Average of Accuracy {}".format(Result_df['Accuracy'].mean()))
print("Average of F1-Score {}".format(Result_df['F1-Score'].mean()))
print("Average of Precision {}".format(Result_df['Precision'].mean()))
print("Average of Recall {}".format(Result_df['Recall'].mean()))

Average of Accuracy 0.9091959953308105
Average of F1-Score 0.926741870934114
Average of Precision 0.956454842807984
Average of Recall 0.9091959972394754


In [10]:
Matrix=pd.DataFrame(Result_df['Accuracy'],columns=['Accuracy'])
Matrix['Accuracy']=Result_df['Accuracy']
A=[Result_df['Accuracy'].mean(),Result_df['F1-Score'].mean(),Result_df['Precision'].mean(),Result_df['Recall'].mean()]
A=pd.DataFrame(A,columns=['Accuracy'])
Matrix=pd.concat([Matrix,A])
Matrix=Matrix.transpose()
Matrix.to_excel('./PFMatrix_Keras_1.xlsx')